<a href="https://colab.research.google.com/github/PierceWind/Data-Scraping/blob/main/activity1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install pandas

In [ ]:
!pip install requests

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# URLs to be scraped
urls = [
    'https://en.wikipedia.org/wiki/Jenna_Ortega',
    'https://disneychannel.fandom.com/wiki/Jenna_Ortega' #has section something
    # This is a placeholder; use the second link here
]

In [ ]:

# Extract information from both URLs
data = [extract_text_from_url(url) for url in urls if extract_text_from_url(url) is not None]

# Create DataFrame for structured information
df_info = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df_info.to_csv('jenna.csv', index=False)

# Print the DataFrame
print(df_info)


                                                 URL  \
0         https://en.wikipedia.org/wiki/Jenna_Ortega   
1  https://disneychannel.fandom.com/wiki/Jenna_Or...   

                                          Early_life  \
0                                 Section not found.   
1  Jenna Ortega was born on September 27, 2002 in...   

                                              Career  \
0                                 Section not found.   
1  Ortega started by making appearances on variou...   

                                         Filmography  \
0                                 Section not found.   
1  Year Title Role Notes 2013 Iron Man 3 Vice Pre...   

                                              Trivia  
0                                 Section not found.  
1  When not busy with work, she enjoys playing so...  


DISNEY CHANNEL

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# URL to be scraped
url = 'https://disneychannel.fandom.com/wiki/Jenna_Ortega'

def extract_text_from_url(url):
    # Send a GET request to the page
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Create BeautifulSoup object; parse with 'html.parser'
        soup = bs(response.text, 'html.parser')

        # Define a function to extract section text
        def extract_section_text(section_id):
            try:
                # Find the section header
                section_header = soup.find('span', {'id': section_id}).find_parent('h2')

                # Initialize content variable
                content = ''

                # Find the next sibling after the section header
                next_element = section_header.find_next_sibling()

                # Extract text from paragraphs, table, and list (if applicable)
                while next_element and not next_element.name.startswith('h'):
                    if next_element.name == 'p':
                        content += next_element.get_text(strip=True) + ' '
                    elif next_element.name == 'ul':
                        list_items = next_element.find_all('li')
                        for li in list_items:
                            content += li.get_text(strip=True) + ' '
                    elif next_element.name == 'table':
                        # Extract text from table rows
                        for row in next_element.find_all('tr'):
                            cells = row.find_all(['td', 'th'])
                            for cell in cells:
                                content += cell.get_text(strip=True) + ' '

                    # Move to the next sibling
                    next_element = next_element.find_next_sibling()

                return content.strip()

            except AttributeError:
                return 'Section not found.'

        # Extracting information from different sections
        sample_text = extract_section_text('Early_life')
        sample2_text = extract_section_text('Career')
        film_text = extract_section_text('Filmography')
        trivia_text = extract_section_text('Trivia')

        # Return extracted text
        return {
            'URL': url,
            'Early_life': sample_text,
            'Career': sample2_text,
            'Filmography': film_text,
            'Trivia': trivia_text
        }
    else:
        print(f"Failed to retrieve data from {url}. Status code: {response.status_code}")
        return None

# Extract information from the URL
data = extract_text_from_url(url)

# Create DataFrame for structured information
df_info = pd.DataFrame([data])

# Save the DataFrame to a CSV file
df_info.to_csv('jenna.csv', index=False)

# Print the DataFrame
print(df_info)


                                                 URL  \
0  https://disneychannel.fandom.com/wiki/Jenna_Or...   

                                          Early_life  \
0  Jenna Ortega was born on September 27, 2002 in...   

                                              Career  \
0  Ortega started by making appearances on variou...   

                                         Filmography  \
0  Year Title Role Notes 2013 Iron Man 3 Vice Pre...   

                                              Trivia  
0  When not busy with work, she enjoys playing so...  


In [ ]:
df_info

,URL,Early_life,Career,Filmography,Trivia
0,https://disneychannel.fandom.com/wiki/Jenna_Or...,"Jenna Ortega was born on September 27, 2002 in...",Ortega started by making appearances on variou...,Year Title Role Notes 2013 Iron Man 3 Vice Pre...,"When not busy with work, she enjoys playing so..."


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# URL to be scraped
url = 'https://en.wikipedia.org/wiki/Jenna_Ortega'

def extract_sections_from_url(url):
    # Send a GET request to the page
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Create BeautifulSoup object; parse with 'html.parser'
        soup = bs(response.content, 'html.parser')

        # Find all the section headers and their content
        sections = {}
        current_section = None

        # Find all headers (h2, h3, etc.)
        for header in soup.find_all(['h2', 'h3']):
            section_name = header.text.strip()
            sections[section_name] = ''
            current_section = section_name

            # Find content between current header and next header or end of page
            next_node = header
            while True:
                next_node = next_node.find_next()

                # If next_node is None or is a header, stop
                if next_node is None or next_node.name in ['h2', 'h3']:
                    break

                # Append text to current section
                if next_node.name == 'p':
                    sections[current_section] += next_node.get_text(strip=True) + ' '

        return sections
    else:
        print(f"Failed to retrieve data from {url}. Status code: {response.status_code}")
        return None


# Extract sections from the URL
sections_data = extract_sections_from_url(url)

# Access content of the "Career" section and its sub-sections
career_content = ''
if 'Career' in sections_data:
    career_content = sections_data['Career']

# Create DataFrame from the extracted sections
df_sections = pd.DataFrame(sections_data.items(), columns=['Section', 'Content'])

# Save the DataFrame to a CSV file
df_sections.to_csv('jenna2.csv', index=False)

# Print the DataFrame
print(df_sections)

# Print the content of the "Career" section
print("\nContent of 'Career' section:\n", career_content)


                                           Section  \
0                                         Contents   
1                                       Early life   
2                                           Career   
3         2012–2017: Early acting roles and Disney   
4            2018–2021: Transition to mature roles   
5   2022–present: Mainstream success and Wednesday   
6                                   Other ventures   
7                                         Artistry   
8                                       Influences   
9                    Acting style and role choices   
10                                    Public image   
11                                         Fashion   
12                                   Personal life   
13                                     Filmography   
14                                            Film   
15                                      Television   
16                                    Music videos   
17                          

WIKIPEDIA

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# URL to be scraped
url = 'https://en.wikipedia.org/wiki/Jenna_Ortega'

def extract_sections_from_url(url):
    # Send a GET request to the page
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Create BeautifulSoup object; parse with 'html.parser'
        soup = bs(response.content, 'html.parser')

        # Find all the section headers and their content
        sections = {
            'URL': url,
            'Early_life': '',
            'Career': '',
            'Filmography': '',
            'Trivia': ''
        }

        # Find all headers (h2, h3, etc.)
        for header in soup.find_all(['h2', 'h3']):
            section_name = header.text.strip()

            if section_name == 'Early life':
                content = header.find_next('p')
                if content:
                    sections['Early_life'] = content.get_text(strip=True)

            elif section_name == 'Other ventures':
                content = ''
                next_node = header
                while True:
                    next_node = next_node.find_next()

                    if next_node is None or next_node.name in ['h2', 'h3']:
                        break

                    if next_node.name == 'p':
                        content += next_node.get_text(strip=True) + ' '

                if content:
                    sections['Career'] = content.strip()  # Populate 'Career' with 'Other ventures' content

            elif section_name == 'Television':
                # Find the first table under the "Film" section with class 'wikitable'
                table = header.find_next('table', class_='wikitable')
                if table:
                    filmography = []
                    headers = [th.text.strip() for th in table.find('tr').find_all('th')]
                    for row in table.find_all('tr')[1:]:
                        filmography.append({headers[i]: cell.text.strip() for i, cell in enumerate(row.find_all('td'))})
                    sections['Filmography'] = filmography
                    break  # Stop processing further headers once Filmography is found

            elif section_name == 'Personal life':
                content = header.find_next('p')
                if content:
                    sections['Trivia'] = content.get_text(strip=True)

        return sections

    else:
        print(f"Failed to retrieve data from {url}. Status code: {response.status_code}")
        return None

# Extract sections from the URL
sections_data = extract_sections_from_url(url)

if sections_data:
    # Create DataFrame from the extracted sections
    df_sections = pd.DataFrame([sections_data])

    # Save the DataFrame to a CSV file
    df_sections.to_csv('jenna2.csv', index=False)

    # Print the required DataFrame format
    print(df_sections[['URL', 'Early_life', 'Career', 'Filmography', 'Trivia']])
else:
    print("Data extraction failed. Check the URL or try again later.")


                                          URL  \
0  https://en.wikipedia.org/wiki/Jenna_Ortega   

                                          Early_life  \
0  Jenna Marie Ortega,[1]the fourth of six siblin...   

                                              Career  \
0  Ortega has used her platform to support variou...   

                                         Filmography  \
0  [{'Year': 'Rob', 'Title': 'Girl', 'Role': 'Epi...   

                                              Trivia  
0  Ortega has said that her friends and family gi...  


In [ ]:
df_sections


,URL,Early_life,Career,Filmography,Trivia
0,https://en.wikipedia.org/wiki/Jenna_Ortega,"Jenna Marie Ortega,[1]the fourth of six siblin...",Ortega has used her platform to support variou...,"[{'Year': 'Rob', 'Title': 'Girl', 'Role': 'Epi...",Ortega has said that her friends and family gi...


In [ ]:
import pandas as pd

# Load the two CSV files into separate DataFrames
df_info = pd.read_csv('jenna.csv')
df_sections = pd.read_csv('jenna2.csv')

# Concatenate the DataFrames
df_combined = pd.concat([df_info, df_sections], ignore_index=True)

# Save the combined DataFrame to a new CSV file
df_combined.to_csv('jenna_combined.csv', index=False)

# Print the combined DataFrame
print(df_combined)


                                          URL  \
0  https://en.wikipedia.org/wiki/Jenna_Ortega   
1  https://en.wikipedia.org/wiki/Jenna_Ortega   

                                          Early_life  \
0  Jenna Marie Ortega,[1]the fourth of six siblin...   
1  Jenna Marie Ortega,[1]the fourth of six siblin...   

                                              Career  \
0  Ortega has used her platform to support variou...   
1  Ortega has used her platform to support variou...   

                                         Filmography  \
0  [{'Year': 'Rob', 'Title': 'Girl', 'Role': 'Epi...   
1  [{'Year': 'Rob', 'Title': 'Girl', 'Role': 'Epi...   

                                              Trivia  
0  Ortega has said that her friends and family gi...  
1  Ortega has said that her friends and family gi...  


In [ ]:
# Load the combined DataFrame
df_combined = pd.read_csv('jenna_combined.csv')

# Check for null values in the DataFrame
null_summary = df_combined.isnull().sum()

# Print the summary of null values
print("Summary of null values in each column:")
print(null_summary)

Summary of null values in each column:
URL            0
Early_life     0
Career         0
Filmography    0
Trivia         0
dtype: int64


In [ ]:
# Save the cleaned DataFrame to a new CSV file
df_combined.to_csv('hinata_cleaned_info.csv', index=False)
df_combined.to_csv('hinata_cleaned_info.txt', '\t', index=False)


# **ACTIVITY 2**